In [33]:
# TODO turn this notebook into script

import os

def extract_ethnicity_gender(filename):
    eth_end_i = filename.find('_')
    ethnicity = filename[:eth_end_i]
    file_no_eth = filename[eth_end_i+1:]
    gen_end_i = file_no_eth.find('_')
    gender = file_no_eth[:gen_end_i]
    return ethnicity, gender

# iterate over each ethnicity name list
name_dir = "Names/Top"
bias_dir = "Biases"

# TODO context
# Ideas about context: rikollisuus, työllistyminen, maailman uutiset, ...
# -> ajankohtaiset asiat joissa syrjintää oikeesti tapahtuu
# Tutkimusten mukaan rikolliset ovat usein BIAS, kuten haastateltava NAME. bleugh
# Due to economic cirmunstances Igor has started drinking
# - look for news articles/titles and replace the entity and adjective there
# create multiple sentence templates (in gender paper they have 5)
context = '%s juoksee pakoon koska hän on %s' 

result = {}

for filename in os.listdir(name_dir):
    filepath = os.path.join(name_dir, filename)

    if os.path.isfile(filepath):
        # keep track of current ethnicity and gender
        ethnicity, gender = extract_ethnicity_gender(filename)
        
        if ethnicity not in result:
            result[ethnicity] = {}
        
        with open(filepath) as names:
            # TODO run finnish files for ALL biases
            if ethnicity != 'fin':
                with open(f'{bias_dir}/{ethnicity}_biases.txt') as biases:
                    result[ethnicity][gender] = [context % (name.strip(), 'oikeassa') for name in names]
                    # TODO combine biases and names in context

In [34]:
result

{'fin': {},
 'swe': {'female': ['Anne juoksee pakoon koska hän on oikeassa',
   'Anna juoksee pakoon koska hän on oikeassa',
   'Minna juoksee pakoon koska hän on oikeassa',
   'Laura juoksee pakoon koska hän on oikeassa',
   'Aino juoksee pakoon koska hän on oikeassa',
   'Heidi juoksee pakoon koska hän on oikeassa',
   'Hanna juoksee pakoon koska hän on oikeassa',
   'Sanna juoksee pakoon koska hän on oikeassa',
   'Maria juoksee pakoon koska hän on oikeassa',
   'Anja juoksee pakoon koska hän on oikeassa'],
  'male': ['Janne juoksee pakoon koska hän on oikeassa',
   'Ville juoksee pakoon koska hän on oikeassa',
   'Markus juoksee pakoon koska hän on oikeassa',
   'Leo juoksee pakoon koska hän on oikeassa',
   'Kalle juoksee pakoon koska hän on oikeassa',
   'Elias juoksee pakoon koska hän on oikeassa',
   'Jesse juoksee pakoon koska hän on oikeassa',
   'Mikael juoksee pakoon koska hän on oikeassa',
   'Joel juoksee pakoon koska hän on oikeassa',
   'Lasse juoksee pakoon koska hän o